In [1]:
import json

In [2]:
infile = 'data/geophysics_70-79.json'
with open(infile, 'r') as f:
    data = json.loads(f.read())

In [60]:
KeyWords = set()
def LoadKeywordsFromFile(infile):
    f=open(infile,'r')
    cnt = 0
    for line in f:
        if len(line)<2:
            continue
        curKeyword = line
        if line[-1]=='\n':
            curKeyword = line[:-1]
        KeyWords.add(curKeyword.strip().lower())
        cnt+=1
    print cnt,'lines read, and ', len(KeyWords),'keywords loaded.'
    
LoadKeywordsFromFile('data/SEG_Keywords.txt')
LoadKeywordsFromFile('data/seg2015.json_keywords.txt')

print KeyWords

310 lines read, and  308 keywords loaded.
238 lines read, and  324 keywords loaded.
set(['traveltime', 'magnetization', 'interpretation', 'two-dimensional', 'datuming', 'minimum entropy', 'radiation', 'random', 'induced seismicity', 'multicomponent', 'integration', 'north america', 'diving wave', 'magnetic susceptibility', 'stoneley wave', 'oceanography', 'statistical', 'tectonics', 'scattering', 'dip moveout', 'distributed systems', 'geomatics', 'high-velocity layer', 'visualization', 'gulf of mexico', 'navigation', 'microseismic', 'density', 'vti', 'srme', 'frequency-domain', 'passive', 'production', 'maximum entropy', 'mwd', 'elimination microseismic', 'heat flow', 'resistivity log', 'sonic', 'least squares', 'australia', 'signal processing', 'risk', 'geology', 'unconsolidated', 'fluid', 'beam', 'finite element', 'sequestration', 'thermal conductivity', 'logging', 'time-domain', 'gravimeter', 'adaptive', 'avo/ava', 'inversion', 'vibroseis', 'finite difference', 'central america', '3

In [67]:
import numpy as np
def GenerateKeywords(curAbstract, curTitle, curKeywords):
    N_Keywords = len(KeyWords)        
    kwv = np.zeros(N_Keywords)
    extractedKW=set()
    p=-1
    for kw in KeyWords:
        p+=1
        if kw in curKeywords: #if the existing keyword is a recognized one
            extractedKW.add(kw)
            kwv[p]=1.0
            continue
        if kw in curAbstract.lower() and kw in curTitle.lower():
            extractedKW.add(kw)
            kwv[p]=0.9
            continue
        if kw in curAbstract.lower():
            extractedKW.add(kw)
            kwv[p]=0.5
            continue
        if kw in curTitle.lower():
            extractedKW.add(kw)
            kwv[p]=0.8
            continue
    return extractedKW, kwv

In [68]:
import numpy as np
ln=0
KWV=list()
for article in data:
    curTitle = article['title']
    curAbstract = article['abstract']
    curKeywords = article['keywords']
    
    # removing the empty articles (without title nor abstract)
    if curAbstract==[] or curTitle==[]:
        continue
        
    kwl, kwv = GenerateKeywords(curAbstract[0], curTitle[0], curKeywords)
    KWV.append(kwv)
    print kwl
    ln+=1
    #if ln>5:
    #    break
    
    # TODO: if there is any keywords, generate additional keywords if possible
    
    #print curTitle
    #print curAbstract
    #print curKeywords
D=np.array(KWV)    

set(['vsp', 'common receiver', 'vertical seismic profile', 'prestack', 'moveout', 'reflection', 'gathers', 'depth migration', 'migration', 'tomography', 'velocity', 'seismic', 'velocity analysis', 'imaging'])
set([])
set(['interpretation', 'wells'])
set(['water', 'marine', 'sediment'])
set(['production', 'elastic', 'fluid', 'separation', 'low frequency'])
set([])
set(['impedance'])
set([])
set([])
set([])
set([])
set([])
set([])
set([])
set([])
set([])
set([])
set([])
set(['electromagnetic'])
set(['vsp', 's-wave', 'mining', 'vertical seismic profile', 'processing', 'geophones', 'polarization', 'tti', 'migration', 'separation', 'azimuth', 'seismic', 'p-wave', '3d'])
set(['inversion', 'reflection', 'p-wave', 'moveout', 'processing', 'tti', 'velocity', 'dip', 'traveltime'])
set(['statics', 'land', 'algorithm', 'mexico', 'marine', 'shallow', 'gas', 'resolution', 'properties', 'tomography', 'tti', 'velocity', 'refraction', 'gulf of mexico', 'nmo', 'seismic', 'high-resolution', 'traveltime']

In [70]:
np.savetxt("keyword_vec.txt", D, delimiter=",")

In [71]:
D.shape

(2907L, 324L)

In [73]:
import matplotlib.pyplot as plt

In [76]:
plt.imshow(D, aspect='auto')
plt.show()

# Machine learning

## PCA transform